In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math
import pandas as pd
import json
from tqdm import tqdm
import ast
import nltk
from nltk.corpus import stopwords
import re

In [29]:
#testing one off
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
input_text = "The quick brown fox jumps over the lazy dog."

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # or bfloat16 or int8/int4 if quantized
    device_map="auto"
)
model.eval()



/ihome/crc/install/pytorch/2.0.1/python3.10/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/ihome/crc/install/pytorch/2.0.1/python3.10/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/ihome/crc/install/pytorch/2.0.1/python3.10/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out ht

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [33]:
#data sources

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


full_df = pd.read_csv('combined_wide_form_jul30.csv')
irdf = pd.read_csv("interpretive_repertoires.csv")

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    tokens = text.split()
    # tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

full_df['text'] = full_df['human_clean_title'] + "\n" + full_df['human_clean_text']
# full_df["clean"] = full_df["raw_text"].apply(clean_text)
full_df["charm_article_id"] = full_df.index + 1

# full_df.head()
fdf = full_df[['charm_article_id', 'source_url',  'text']].copy()

# fdf = fdf.head(2) #for testing

[nltk_data] Downloading package stopwords to
[nltk_data]     /ihome/xli/sek188/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
#split on paragraphs:
pdf = pd.DataFrame(columns=["charm_article_id", "text", "position"])
for idx, row in fdf.iterrows():
    paras = row["text"].split('\n')
    idx = 0
    #remove any single line paragraphs (common in news articles)
    while idx < len(paras):
        num_sentences = len([sent for sent in re.split(r'[.!?]', paras[idx]) if sent.strip()])
        if num_sentences < 2 and idx > 0:
            paras[idx-1] += ' ' + paras[idx]
            s = paras.pop(idx)
        else:
            idx += 1

    cparas = []
    for position, para in enumerate(paras):
        newpara  = {
            "charm_article_id": row["charm_article_id"],
            "text": para.strip(),
            "position": position + 1
        }
        cparas.append(newpara)
    pdf = pd.concat([pdf, pd.DataFrame(cparas)], ignore_index=True)

pdf["text"] = pdf["text"].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()

In [35]:
#stitching and liloing
combined_rows = []
for _, f_row in pdf.iterrows():
    for _, ir_row in irdf.iterrows():
        combined_row = {
            **f_row.to_dict(),      # all columns from fdf
            **ir_row.to_dict()      # all columns from irdf
        }
        combined_rows.append(combined_row)

cdf = pd.DataFrame(combined_rows)
cdf["attestation"] = cdf["likes"].combine_first(cdf["dislikes"])
cdf["input_text"] = cdf["attestation"] + "\n\n" + cdf["text"]
cdf.drop(columns=['likes', 'dislikes', 'guidewords'])

funkyoutputblocker = None

In [36]:
logprobs = []
for idx, row in tqdm(cdf.iterrows(), total=len(paras)):
    inputs = tokenizer(row['text'], return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    input_ids = inputs["input_ids"]
    shift_logits = logits[:, :-1, :]
    shift_labels = input_ids[:, 1:]
    log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
    token_log_probs = log_probs.gather(2, shift_labels.unsqueeze(-1)).squeeze(-1)
    average_log_prob = token_log_probs.mean().item()
    logprobs.append(average_log_prob)

cdf["logprob"] = logprobs

70056it [25:15, 46.22it/s]                     


In [31]:
cdf.head()

,charm_article_id,text,position,IR,bucket,guidewords,likes,dislikes,attestation,input_text,logprob
0,1,trump says 'male circumcision in mozambique' i...,1,Inspired,+,"['righteous', 'pre-ordained', 'beautiful']",I am most deeply moved by things that feel rig...,NaN,I am most deeply moved by things that feel rig...,I am most deeply moved by things that feel rig...,-3.748047
1,1,trump says 'male circumcision in mozambique' i...,1,Inspired,-,"['false', 'uncreative', 'dull']",NaN,I have a visceral reaction to anything that st...,I have a visceral reaction to anything that st...,I have a visceral reaction to anything that st...,-3.748047
2,1,trump says 'male circumcision in mozambique' i...,1,Popular,+,"['preferred', 'popular', 'favorite']",I take comfort in knowing what’s preferred by ...,NaN,I take comfort in knowing what’s preferred by ...,I take comfort in knowing what’s preferred by ...,-3.748047
3,1,trump says 'male circumcision in mozambique' i...,1,Popular,-,"['resented', 'feared', 'isolated']",NaN,What unsettles me most is when something or so...,What unsettles me most is when something or so...,What unsettles me most is when something or so...,-3.748047
4,1,trump says 'male circumcision in mozambique' i...,1,Moral,+,"['solidary', 'responsible', 'just']",I’m most at ease when I feel like people are l...,NaN,I’m most at ease when I feel like people are l...,I’m most at ease when I feel like people are l...,-3.748047


In [37]:
cdf.to_csv('cohcharm_longlong_jul30.csv')